Xây dựng mô hình dự báo thời gian trễ chuyến bay đến (Arrival Delay) dựa trên thông tin khởi hành và vận hành mặt đất

Làm sạch dữ liệu

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [42]:
df = pd.read_csv(r"C:\Users\USER\PyCharmMiscProject\dataset\flights_sample_3m.csv")
df

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2022-11-13,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,1522,JAX,"Jacksonville, FL",CLT,"Charlotte, NC",...,0.0,85.0,71.0,55.0,328.0,NaN,NaN,NaN,NaN,NaN
2999996,2022-11-02,American Airlines Inc.,American Airlines Inc.: AA,AA,19805,1535,ORD,"Chicago, IL",AUS,"Austin, TX",...,0.0,176.0,145.0,130.0,977.0,NaN,NaN,NaN,NaN,NaN
2999997,2022-09-11,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2745,HSV,"Huntsville, AL",ATL,"Atlanta, GA",...,0.0,55.0,50.0,28.0,151.0,0.0,36.0,0.0,0.0,0.0
2999998,2019-11-13,Republic Airline,Republic Airline: YX,YX,20452,6134,BOS,"Boston, MA",LGA,"New York, NY",...,0.0,88.0,77.0,50.0,184.0,NaN,NaN,NaN,NaN,NaN


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 int64  
 5   FL_NUMBER                int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             int64  
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             int64  
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  CANCELLA

In [44]:
print(df.isna().sum())

FL_DATE                          0
AIRLINE                          0
AIRLINE_DOT                      0
AIRLINE_CODE                     0
DOT_CODE                         0
FL_NUMBER                        0
ORIGIN                           0
ORIGIN_CITY                      0
DEST                             0
DEST_CITY                        0
CRS_DEP_TIME                     0
DEP_TIME                     77615
DEP_DELAY                    77644
TAXI_OUT                     78806
WHEELS_OFF                   78806
WHEELS_ON                    79944
TAXI_IN                      79944
CRS_ARR_TIME                     0
ARR_TIME                     79942
ARR_DELAY                    86198
CANCELLED                        0
CANCELLATION_CODE          2920860
DIVERTED                         0
CRS_ELAPSED_TIME                14
ELAPSED_TIME                 86198
AIR_TIME                     86198
DISTANCE                         0
DELAY_DUE_CARRIER          2466137
DELAY_DUE_WEATHER   

In [45]:
df = df[[
    "FL_DATE",
    "AIRLINE_CODE",
    "FL_NUMBER",
    "ORIGIN",
    "DEST",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "DISTANCE",
    "ARR_DELAY"
]]


In [46]:
df

,FL_DATE,AIRLINE_CODE,FL_NUMBER,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,DISTANCE,ARR_DELAY
0,2019-01-09,UA,1562,FLL,EWR,1155,1501,1065.0,-14.0
1,2022-11-19,DL,1149,MSP,SEA,2120,2315,1399.0,-5.0
2,2022-07-22,UA,459,DEN,MSP,954,1252,680.0,0.0
3,2023-03-06,DL,2295,MSP,SFO,1609,1829,1589.0,24.0
4,2020-02-23,NK,407,MCO,DFW,1840,2041,985.0,-1.0
...,...,...,...,...,...,...,...,...,...
2999995,2022-11-13,AA,1522,JAX,CLT,1742,1907,328.0,-16.0
2999996,2022-11-02,AA,1535,ORD,AUS,1300,1556,977.0,-37.0
2999997,2022-09-11,DL,2745,HSV,ATL,534,729,151.0,36.0
2999998,2019-11-13,YX,6134,BOS,LGA,1600,1728,184.0,-16.0


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   FL_DATE       object 
 1   AIRLINE_CODE  object 
 2   FL_NUMBER     int64  
 3   ORIGIN        object 
 4   DEST          object 
 5   CRS_DEP_TIME  int64  
 6   CRS_ARR_TIME  int64  
 7   DISTANCE      float64
 8   ARR_DELAY     float64
dtypes: float64(2), int64(3), object(4)
memory usage: 206.0+ MB


In [49]:
print(df.isna().sum())

FL_DATE             0
AIRLINE_CODE        0
FL_NUMBER           0
ORIGIN              0
DEST                0
CRS_DEP_TIME        0
CRS_ARR_TIME        0
DISTANCE            0
ARR_DELAY       86198
dtype: int64


Có 86198 dòng ARR_DELAY bị NaN trên tổng số 3 triệu dòng dữ liệu của dataset -> Quá ít -> Xóa luôn

In [50]:
df = df.dropna(subset=["ARR_DELAY"])

Tiền xử lí dữ liệu

In [ ]:
df["FL_DATE"] = pd.to_datetime(df["FL_DATE"])

In [53]:
df["YEAR"] = df["FL_DATE"].dt.year
df["MONTH"] = df["FL_DATE"].dt.month
df["DAY"] = df["FL_DATE"].dt.day
df["WEEKDAY"] = df["FL_DATE"].dt.weekday

C:\Users\USER\AppData\Local\Temp\ipykernel_10720\2172852486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["YEAR"] = df["FL_DATE"].dt.year
C:\Users\USER\AppData\Local\Temp\ipykernel_10720\2172852486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MONTH"] = df["FL_DATE"].dt.month
C:\Users\USER\AppData\Local\Temp\ipykernel_10720\2172852486.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [54]:
df

,FL_DATE,AIRLINE_CODE,FL_NUMBER,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,DISTANCE,ARR_DELAY,YEAR,MONTH,DAY,WEEKDAY
0,2019-01-09,UA,1562,FLL,EWR,1155,1501,1065.0,-14.0,2019,1,9,2
1,2022-11-19,DL,1149,MSP,SEA,2120,2315,1399.0,-5.0,2022,11,19,5
2,2022-07-22,UA,459,DEN,MSP,954,1252,680.0,0.0,2022,7,22,4
3,2023-03-06,DL,2295,MSP,SFO,1609,1829,1589.0,24.0,2023,3,6,0
4,2020-02-23,NK,407,MCO,DFW,1840,2041,985.0,-1.0,2020,2,23,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2022-11-13,AA,1522,JAX,CLT,1742,1907,328.0,-16.0,2022,11,13,6
2999996,2022-11-02,AA,1535,ORD,AUS,1300,1556,977.0,-37.0,2022,11,2,2
2999997,2022-09-11,DL,2745,HSV,ATL,534,729,151.0,36.0,2022,9,11,6
2999998,2019-11-13,YX,6134,BOS,LGA,1600,1728,184.0,-16.0,2019,11,13,2


Chuyển CRS_DEP_TIME / CRS_ARR_TIME sang phút trong ngày

In [55]:
def convert_time(t):
    t = int(t)
    return (t // 100) * 60 + (t % 100)

df["DEP_MIN"] = df["CRS_DEP_TIME"].apply(convert_time)
df["ARR_MIN"] = df["CRS_ARR_TIME"].apply(convert_time)

C:\Users\USER\AppData\Local\Temp\ipykernel_10720\4196097718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DEP_MIN"] = df["CRS_DEP_TIME"].apply(convert_time)
C:\Users\USER\AppData\Local\Temp\ipykernel_10720\4196097718.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ARR_MIN"] = df["CRS_ARR_TIME"].apply(convert_time)


Xóa cột không cần nữa

In [56]:
df = df.drop(columns=["CRS_DEP_TIME", "CRS_ARR_TIME", "FL_DATE"])

EDA (khám phá dữ liệu)